In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#install packages
install.packages("dplyr")
install.packages("lubridate")
install.packages('tidyverse')
install.packages('skimr')
install.packages("ggplot2")


In [2]:
#Load packages
library(dplyr)
library(lubridate)
library(skimr)
library(tidyverse)
library(ggplot2)

In [3]:
#Getting the data
April2020_trip<-read.csv("../input/cyclistics-data-april-2020-march-2021/202004-divvy-tripdata.csv")
May2020_trip<-read.csv('../input/cyclistics-data-april-2020-march-2021/202005-divvy-tripdata.csv')
June2020_trip<-read.csv('../input/cyclistics-data-april-2020-march-2021/202006-divvy-tripdata.csv')
July2020_trip<-read.csv('../input/cyclistics-data-april-2020-march-2021/202007-divvy-tripdata.csv')
August2020_trip<-read.csv('../input/cyclistics-data-april-2020-march-2021/202008-divvy-tripdata.csv')
Septmber2020_trip<-read.csv('../input/cyclistics-data-april-2020-march-2021/202009-divvy-tripdata.csv')
October2020_trip<-read.csv('../input/cyclistics-data-april-2020-march-2021/202010-divvy-tripdata.csv')
November2020_trip<-read.csv('../input/cyclistics-data-april-2020-march-2021/202011-divvy-tripdata.csv')
December2020_trip<-read.csv('../input/cyclistics-data-april-2020-march-2021/202012-divvy-tripdata.csv')
Januar2021_trip<-read.csv('../input/cyclistics-data-april-2020-march-2021/202101-divvy-tripdata.csv')
Februar2021_trip<-read.csv('../input/cyclistics-data-april-2020-march-2021/202102-divvy-tripdata.csv')
March2021_trip<-read.csv('../input/cyclistics-data-april-2020-march-2021/202103-divvy-tripdata.csv')
April2021_trip<-read.csv('../input/cyclistics-data-april-2020-march-2021/202104-divvy-tripdata.csv')

In [4]:
#listing it before Join.
Year_data<- list(April2020_trip,May2020_trip,June2020_trip,July2020_trip,August2020_trip,Septmber2020_trip,October2020_trip,November2020_trip,December2020_trip,Januar2021_trip,Februar2021_trip,March2021_trip,April2021_trip)


In [5]:
Merged<-Year_data%>%reduce(left_join, by="ride_id")

In [6]:
# Getting what I need from the table
Merged<- subset(Merged, select = c(ride_id,rideable_type.x,started_at.x,ended_at.x,start_station_name.x,start_station_id.x,end_station_name.x,end_station_id.x,start_lat.x,start_lng.x,end_lat.x,end_lng.x,member_casual.x))

In [7]:
str(Merged)

In [8]:
#Converting all columns to Character
Merged<- lapply(Merged,as.character)

In [9]:
#Setting dates
Merged$started_at.x<- as_datetime(Merged$started_at.x)
Merged$ended_at.x <- as_datetime(Merged$ended_at.x)

In [10]:
str(Merged)

In [11]:
#Getting some insights about the data
Merged$duration<-with(Merged,difftime(Merged$ended_at.x,Merged$started_at.x,units="mins"))

In [12]:
Merged<-as.data.frame(Merged)
Merged$duration<-as.numeric(Merged$duration)

In [13]:
#Looking for issues related to the duration
Merged[with(Merged,duration>500),]
Ds<-Merged[with(Merged,duration<0),]

In [14]:
#There's a problem with some rides, the return time is before take
Ds[order(Ds$start_station_id.x),,drop=FALSE]

Return time after take time: 
There are 51(13.7%) incidents where 44 are not the same station. (86.3%)
It is not related to distance nor 


In [24]:
#Renting a bike for longer periods seems odd, specially if it is longer than 8 hours. 
count(Merged[with(Merged,duration>720),])

It seems like there are 231 cases of rentals that were longer than 12 hours. The longest was 40 days... 
For this data case, my assumption is that a person will not rent a bike for longer than 12 hours. 

In [30]:
clean2020_march2021<-Merged[with(Merged,duration<720 & duration>=0),]
write.csv(clean2020_march2021,"../kaggle/working/MA20_21.csv"